This tutorial deals with reading an SU data set and __creating a SEG-Y file__. Please make sure you have downloaded the test data as outlined in the file "examples/data/README" and you ran tutorial #2 to create the required SU file.

Let's start by importing some packages (including the seisio package itself) and setting up the logging config.

In [1]:
import logging
import numpy as np

import sys
sys.path.insert(1, '../')

In [2]:
import seisio

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)
log = logging.getLogger("main")

We read the (previously created) SU file in little-endian byte order and create a SEG-Y file, but as opposed to the original 'Line_001.sgy' file we use the IEEE floating-point data format.

In [4]:
infile = "data/Line_001.su"          # SU file, IEEE floats, little endian
outfile = "data/Line_001_IEEE.sgy"   # SEGY file, IEEE floats, big endian

Let's create the seisio object to read the input SU file...

In [5]:
sin = seisio.input(infile)

INFO: Input file: data/Line_001.su
INFO: Assuming fixed-length traces for SU data.
INFO: Data sample format: 4-byte IEEE floating-point.
INFO: Input file endianess looks to be '<' (best guess).
INFO: Byte offset of first trace relative to start of file: 0 bytes.
INFO: Number of samples per data trace: 1501.
INFO: Sampling interval: 2000 (unit as per SU standard).
INFO: Delay (on first trace): 0 (unit as per SU standard).
INFO: Number of data traces in file: 71284.


...and the seisio object to write the SEG-Y file in big-endian byte order using format 5 (IEEE floats). The primary textual file header should be encoded as EBCDIC.

In [6]:
sout = seisio.output(outfile, ns=sin.ns, vsi=sin.vsi, endian=">", format=5, txtenc="ebcdic")

INFO: Output file: data/Line_001_IEEE.sgy
INFO: Output number of samples per data trace: 1501.
INFO: Output data sample format: 4-byte IEEE floating-point.
INFO: Output file endianess set to '>'.
INFO: Number of additional textual header records: 0.
INFO: Byte offset of first trace relative to start of file: 3600 bytes.
INFO: Number of trailer stanza records: 0.
INFO: SEG-Y trace header extension 1 is not present.
INFO: Number of user-defined trace headers: 0
INFO: Creating file according to SEG-Y rev. 1.0.


SEG-Y files require (at least) a primary textual file header (3200 bytes) and a binary file header (400 bytes). We can get templates and fill them in ourselves, or we let seisio create default (minimal) file headers for us similar to SU's _segyhdrs_ program.

In [7]:
textual_template = sout.txthead_template
binary_template = sout.binhead_template

We could now modify or update those templates and use them for the output SEG-Y file. However, here we let seisio create default headers. In this case, we have no arguments for the *init()* method that we need to call before actually writing any trace data. The method writes the textual and binary file headers and, where applicable, additional textual header records. Here, we have no additional header records.

In [8]:
sout.init()

INFO: SEG-Y textual file header encoding set to 'EBCDIC'.
INFO: Wrote textual and binary file headers and 0 add. header record(s).


You can check the size of the output file at any point. Now that we have written the SEG-Y file headers, the file size should be 3600 bytes.

In [9]:
log.info("Size of output file now %d bytes (should be 3600).", sout.fsize)

INFO: Size of output file now 3600 bytes (should be 3600).


Calling the *init()* method several times is ignored:

In [10]:
sout.init()

Now we read the whole data set from the SU file but we could obviously also loop through the data in case of very large files.

In [11]:
dataset = sin.read_dataset()

INFO: Reading entire file (71284 traces) from disk...
INFO: Reading all traces took 0.1 seconds.


Before writing the data to disk, let's sort the data in some way (just for demonstrating the feature) in descending order:

In [12]:
dataset = np.sort(dataset, order=["tracl"])[::-1]

Now we write the data to disk (in the SEG-Y file):

In [13]:
nwritten = sout.write_traces(traces=dataset)
# Some specific SU trace headers (like 'd1', 'd2', etc.) are not transferred.

INFO: Swapping bytes of output data.
INFO: Writing 71284 trace(s) to disk...


Some SU-specific traces headers (like `d1` etc.) are not available in SEG-Y, i.e., those headers have been dropped. We could remap the header mnemonics to valid SEG-Y trace header mnemonics on output in case we wanted to keep the corresponding header values.

Finally, we need to call the *finalize()* method. This method has to be called last, i.e., after writing the last trace data. The method writes any trailer stanza records (if applicable) and updates the binary file header to reflect the correct number of traces, header records and/or trailers in the file.

In [14]:
sout.finalize()

INFO: Finalizing output file and re-writing updated binary header.
INFO: Wrote a total of 71284 trace(s), file size: 445100896 bytes.


Attempting to write additional data to the file after calling the *finalize()* method leads to an exception:

In [15]:
try:
    sout.write_traces(traces=dataset)
except RuntimeError:
    log.info("Caught 'RuntimeError' exception as expected.")

INFO: Caught 'RuntimeError' exception as expected.


The output SEGY file should have the same file size as the original 'Line_001.sgy' file. However, the content of the textual file header will be different, the data sample format will now be IEEE, the `cdp` header has been filled, the coordinate and elevation scaler has been
zeroed, the traces are written in a different order, and some trace headers might have been lost due to the fact that SU and SEG-Y use different trace header definitions. You can check the data with the following SU commands:

```
$> segyread tape=data/Line_001_IEEE.sgy | segyclean  | surange
71284 traces:
tracl    1 71284 (1 - 71284)
tracr    1 71284 (1 - 71284)
fldr     231 481 (231 - 481)
tracf    -1 282 (-1 - 282)
ep       32 282 (32 - 282)
cdp      231 481 (231 - 481)
cdpt     1 284 (1 - 284)
trid     0 1 (0 - 1)
nhs      1
counit   3
ns       1501
dt       2000
gain     3
igc      1
afilf    207
afils    298
hcf      207
hcs      298
year     1998
$> cat header
...
$> bhedtopar < binary
jobid=0
lino=0
reno=0
ntrpr=0
nart=0
hdt=2000
dto=0
hns=1501
nso=0
format=5
...
$> segyread tape=data/Line_001.sgy | sumax
5.109500e+03 -1.546142e+03
$> segyread tape=data/Line_001_IEEE.sgy | sumax
5.109500e+03 -1.546142e+03
$>
```
